In [2]:
import torch as t
import sys; sys.path.append("..")
from tpp_trace import *
import utils as u
import tensor_ops as tpp


In [46]:
# mm does the element sum 
def basic_mm(X, Y) :
    lme = t.mm(X.exp(), Y.exp()).log()
    return lme


def lme(X, Y) :
    lme = t.mm(X.exp(), Y.exp()).log()
    correction = t.log(t.ones((), device=X.device)*X.size(1))
    return lme - correction


def logmmmeanexp(X, Y):
    xmax = X.max(dim=1, keepdim=True)[0]
    ymax = Y.max(dim=0, keepdim=True)[0]
    X = X - xmax
    Y = Y - ymax
    
    # NB: need t.matmul instead if broadcasting
    log_exp_prod = t.mm(X.exp(), Y.exp()).log()
    
    return log_exp_prod \
            - t.log(t.ones((), device=xmax.device)*X.size(1)) \
            + xmax + ymax

def max_k(T, k) :    
    return T.max(dim=k, keepdim=True)[0]

def denominator(T, i) :
    return t.ones((), device=T.device) * T.size(i)

def logmulmeanexp(X, Y, dim, centre=True):
    """
        :param X: tensor of log probabilities
        :param Y: tensor of log probabilities, possibly placeholder
        :param dim: dimension to average over
        
        X and Y have their names aligned beforehand
    """
    assert(X.names == Y.names)
    
    if centre:
        xmax = max_k(X, dim)           
        ymax = max_k(Y, dim)
        print(xmax, ymax)
        X = X - xmax
        Y = Y - ymax
    
    # matmul happens in probability space, not log space
    # hence exp first
    log_exp_prod = (X.exp() * Y.exp()) \
                    .sum(dim, keepdim=True) \
                    .log()
    log_size = t.log(denominator(X, dim))
    means = log_exp_prod - log_size
    
    if centre : 
        means += xmax + ymax
        
    return means

In [37]:
X = t.Tensor([[2,2],\
              [3,2]])
Y = t.Tensor([[2,4],\
              [1,1]])

print(t.mm(X, Y), "\n", 
      X*Y, "\n",
      (X*Y).sum(dim=0), "\n", 
      (X*Y).sum(dim=1) )

2*2 + 2*1, \
2*4 + 2*1, \
3*2 + 2*1, \
3*4 + 2*1

tensor([[ 6., 10.],
        [ 8., 14.]]) 
 tensor([[4., 8.],
        [3., 2.]]) 
 tensor([ 7., 10.]) 
 tensor([12.,  5.])


(6, 10, 8, 14)

In [44]:
X = t.Tensor([[.2,.1],\
              [.1,.3]])
X = X.refine_names('_k__a', '_k__b')
Y = t.Tensor([[.5,.3],\
              [.2,.1]])
Y = Y.refine_names('_k__a', '_k__b')

Xl = t.log(X)
Yl = t.log(Y)


X3 = t.Tensor([
                [[1,1],[1,3]],
                [[1,1],[1,3]]
             ])
X3 = X3.refine_names('_k__a', '_k__b', '_k__c')
Y3 = t.Tensor([
                [[2,1],[1,3]],
                [[1,1],[1,5]]
             ])
Y3 = Y3.refine_names('_k__a', '_k__b', '_k__c')


d = {"a":X, "b":Y}
dim = "_k__a"



In [48]:
logmulmeanexp(X3, Y3, dim)

tensor([[[1., 1.],
         [1., 3.]]], names=('_k__a', '_k__b', '_k__c')) tensor([[[2., 1.],
         [1., 5.]]], names=('_k__a', '_k__b', '_k__c'))


tensor([[[2.6201, 2.0000],
         [2.0000, 7.4338]]], names=('_k__a', '_k__b', '_k__c'))

In [26]:
print(basic_mm(X, Y)), 
print(lme(X, Y)), 
print(logmmmeanexp(X, Y))
t.mm(X,Y)

tensor([[4.3133, 6.0486],
        [5.1269, 7.0181]])
tensor([[3.6201, 5.3554],
        [4.4338, 6.3250]])
tensor([[3.6201, 5.3554],
        [4.4338, 6.3250]])


tensor([[ 6., 10.],
        [ 8., 14.]])

In [30]:
import imp
imp.reload(u)



X = t.Tensor([[.2,.1],\
              [.1,.3]])
X = X.refine_names('_k__a', '_k__b')
Y = t.Tensor([[.5,.3],\
              [.2,.1]])
Y = Y.refine_names('_k__a', '_k__b')


print(u.logmulmeanexp(X, Y, dim), "\n\n", 
      u.logmmmeanexp(X, Y))


X = t.Tensor([[.2,.2],\
              [.1,.3]])
X = X.refine_names('_k__a', '_k__b')
Y = t.Tensor([[.5,.3],\
              [.2,.1]])
Y = Y.refine_names('_k__a', '_k__b')


print( "\n\n", u.logmulmeanexp(X, Y, dim), "\n\n", 
      u.logmmmeanexp(X, Y))

tensor([[0.5199, 0.4000]], names=('_k__a', '_k__b')) 

 tensor([[0.5199, 0.3612],
        [0.5512, 0.4000]], names=('_k__a', '_k__b'))


 tensor([[0.5199, 0.4512]], names=('_k__a', '_k__b')) 

 tensor([[0.5612, 0.4050],
        [0.5512, 0.4000]], names=('_k__a', '_k__b'))


In [27]:
X = t.Tensor([[.3,.1],\
              [.1,.3]])
X = X.refine_names('_k__a', '_k__b')
Y = t.Tensor([[.3,.7],\
              [.2,.3]])
Y = Y.refine_names('_k__a', '_k__b')


dim = "_k__a"


# to preserve all dimensions
def logmulmeanexp_exact(X, Y, dim) :
    eXeY = (X.exp() * Y.exp())
    nondims = set(X.names) - set([dim])

    for other_dim in nondims :
        print(other_dim, eXeY.mean(other_dim, keepdim=True).log())

    return eXeY.mean(dim, keepdim=True) \
                        .log()

logmulmeanexp_exact(X, Y, dim), logmmmeanexp(X, Y)

_k__b tensor([[0.7050],
        [0.4612]], names=('_k__a', '_k__b'))


/home/gleech/anaconda3/lib/python3.6/site-packages/torch/tensor.py:605: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:840.)
  return super(Tensor, self).refine_names(names)


(tensor([[0.4612, 0.7050]], names=('_k__a', '_k__b')),
 tensor([[0.4612, 0.7443],
         [0.4512, 0.7050]], names=('_k__a', '_k__b')))

(tensor([[3.6201, 5.3554],
         [4.4338, 6.3250]]), tensor([[3.6201, 5.3554],
         [4.4338, 6.3250]]))